In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from loguru import logger
from dotenv import load_dotenv
from pydantic import BaseModel
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.sparse as sparse
import torch.optim as optim

import numpy as np # required for the scikit-learn pipeline to work
import pandas as pd
import plotly.express as px
import mlflow

load_dotenv()

sys.path.insert(0, '..')

from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L5 - Reco Algo"
    run_name: str = '013-content-based'
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = 'user_id'
    item_col: str = 'parent_asin'
    rating_col: str = 'rating'
    
    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        
        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            import mlflow

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self
    
args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-15 12:16:28.578 | INFO     | __main__:init:28 - Setting up MLflow experiment FSDS RecSys - L5 - Reco Algo - run 013-content-based...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L5 - Reco Algo",
  "run_name": "013-content-based",
  "notebook_persist_dp": "/home/dvquys/frostmourne/reco-algo/notebooks/data/013-content-based",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
from src.train_utils import train, MetricLogCallback
from src.model import ContentBased
import joblib

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# device = 'cpu'
logger.info(f"Using {device} device")

2024-09-15 12:16:28.883 | INFO     | __main__:<module>:9 - Using cuda device


In [6]:
item_metadata_pipeline = joblib.load('../data/item_metadata_pipeline.joblib')

# Test implementation

In [7]:
# Mock data
user_ids = [0, 0, 1, 2, 2]
item_ids = [0, 1, 2, 3, 4]
ratings = [1, 4, 5, 3, 2]
timestamps = [0, 1, 2, 3, 4]
main_category = ['All Electronics', 'Video Games', 'All Electronics', 'Video Games', "Unknown"]
title = ['All Electronics', 'Video Games', 'All Electronics', 'Video Games', "Unknown"]
description = [[], [], ["Video games blah blah"], [], ["blah blah"]]
categories = [[], ["Headsets"], ["Video Games"], [], ["blah blah"]]
price = ["from 14.99", "14.99", "price: 9.99", "20 dollars", "None"]

train_df = pd.DataFrame({
    "user_ids": user_ids,
    "item_ids": item_ids,
    "ratings": ratings,
    "timestamps": timestamps,
    "main_category": main_category,
    "title": title,
    "description": description,
    "categories": categories,
    "price": price,
})
# Drop duplicated item features so that the ContentBased model will fit correctly in terms of index mapping
fit_df = train_df.drop_duplicates(subset=['item_ids'])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)

val_user_ids = [0, 1, 2]
val_item_ids = [2, 1, 2]
val_ratings = [2, 4, 5]
val_timestamps = [5, 6, 7]
val_main_category = ['All Electronics', 'Video Games', 'All Electronics']
val_title = ['All Electronics', 'Video Games', 'All Electronics']
val_description = [["Video games blah blah"], [], ["Video games blah blah"]]
val_categories = [["Video Games"], ["Headsets"], ["Video Games"]]
val_price = ["price: 9.99", "14.99", "price: 9.99"]

val_df = pd.DataFrame({
    "user_ids": val_user_ids,
    "item_ids": val_item_ids,
    "ratings": val_ratings,
    "timestamps": val_timestamps,
    "main_category": val_main_category,
    "title": val_title,
    "description": val_description,
    "categories": val_categories,
    "price": val_price,
})
val_item_features = item_metadata_pipeline.transform(val_df).astype(np.float32)

In [8]:
n_users = len(set(user_ids))
n_items = len(set(item_ids))

print("Mock User IDs:", user_ids)
print("Mock Item IDs:", item_ids)
print("Ratings:", ratings)

model = ContentBased(item_features=train_item_features.todense(), device=device)

items1 = [1, 2]
items2 = [0, 3]
predictions = model.predict(items1, items2)
print(predictions)

print("\n\n")

users = [1]
users_last_item = 2
recommendations = model.recommend([users_last_item], k=args.top_K, progress_bar_type='tqdm_notebook')
print(recommendations)

Mock User IDs: [0, 0, 1, 2, 2]
Mock Item IDs: [0, 1, 2, 3, 4]
Ratings: [1, 4, 5, 3, 2]
tensor([7.4894e-05, 5.7673e-05], device='cuda:0')





Generating Recommendations:   0%|          | 0/1 [00:00<?, ?it/s]

{'item_indice': [2, 2, 2, 2], 'recommendation': [0, 3, 1, 4], 'score': [0.5773647427558899, 5.767273250967264e-05, 4.322697714087553e-05, 0.0]}


# Prep data

In [9]:
train_df = pd.read_parquet("../data/train_item_features.parquet")
val_df = pd.read_parquet("../data/val_item_features.parquet")
val_timestamp = 1628643414042  # https://amazon-reviews-2023.github.io/data_processing/5core.html
full_df = pd.concat([train_df, val_df], axis=0)

In [10]:
user_col = 'user_id'
item_col = 'parent_asin'
timestamp_col = 'timestamp' 
rating_col = 'rating'

In [11]:
from src.id_mapper import IDMapper

In [12]:
user_ids = full_df['user_id'].values
item_ids = full_df['parent_asin'].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-15 12:16:29.784 | INFO     | __main__:<module>:6 - len(unique_user_ids)=12,397, len(unique_item_ids)=5,429


In [13]:
idm = IDMapper()
idm.fit(unique_user_ids, unique_item_ids)

In [14]:
def map_indices(df):
    return (
        df
        .assign(
            user_index=df[user_col].apply(idm.get_user_index),
            item_index=df[item_col].apply(idm.get_item_index),
        )
    )

In [15]:
full_df = full_df.pipe(map_indices)
full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price,user_index,item_index
0,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0863MT183,4.0,1613701986538,All Electronics,KIWI design VR Stand Compatible with Quest 2/R...,[],"[Video Games, PC, Virtual Reality, Headsets]",19.99,5219,946
1,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B08P8P7686,5.0,1613702112995,All Electronics,Seltureone Accessories Compatible for Quest 2 ...,[],"[Video Games, PC, Virtual Reality, Headsets]",20.99,5219,1238
2,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0B7LV3DN2,4.0,1617641445475,Cell Phones & Accessories,Eyglo Replacement Elite Strap for Oculus/Meta ...,[],"[Video Games, PC, Virtual Reality, Headsets]",28.49,5219,3858
3,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B09WMQ6DXG,5.0,1620231368468,All Electronics,"AMVR Controller Grips for Quest 2, Controller ...",[],"[Video Games, PC, Virtual Reality, Controllers]",15.99,5219,1276
4,AFFZVSTUS3U2ZD22A2NPZSKOCPGQ,B01GW3LRD2,5.0,1491589434000,Video Games,Forza Horizon 3 – Xbox One,[You’re in charge of the Horizon Festival. Cus...,"[Video Games, Xbox One, Games]",22.0,1305,2448
...,...,...,...,...,...,...,...,...,...,...,...
15252,AEUDRWR7PNA6JPXZ5RTN6KCXKOBA,B0BK673BF4,5.0,1636340286500,Computers,HK GAMING GK61 Mechanical Gaming Keyboard 60 P...,[],"[Video Games, PC, Accessories, Gaming Keyboards]",59.99,3947,5277
15253,AFEYPRZTCVN4WURYQOEYNTD2JFYQ,B000N5Z2L4,5.0,1648670003849,Video Games,Xbox Live Gold: 1 Month Membership [Digital Code],"[""Experience the rush of competitive and coope...","[Video Games, Xbox Digital Content, Subscripti...",9.99,4970,2621
15254,AHPWCVRF23GACCNGHT6VRIPFTFFA,B0B5SV7L99,5.0,1631554680324,Video Games,Streets of Rage 4 - Xbox One,"[Streets of Rage, considered One of the best b...","[Video Games, Xbox One, Games]",34.99,5883,957
15255,AGRQ2ELNB47RERPPMRDMXK7EOGZA,B0771371PM,5.0,1630392097798,Video Games,"Hyperkin ""Scout"" Premium Controller for SNES","[The Hyperkin ""Scout"" Premium Controller for S...","[Video Games, Legacy Systems, Nintendo Systems...",14.99,10840,2428


# Recommend

In [16]:
from src.dataset_loader import ItemSequenceDataset
from torch.utils.data import DataLoader

In [17]:
fit_df = train_df.drop_duplicates(subset=[args.item_col])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)
model = ContentBased(item_features=train_item_features.todense(), device=device)

In [18]:
full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price,user_index,item_index
0,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0863MT183,4.0,1613701986538,All Electronics,KIWI design VR Stand Compatible with Quest 2/R...,[],"[Video Games, PC, Virtual Reality, Headsets]",19.99,5219,946
1,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B08P8P7686,5.0,1613702112995,All Electronics,Seltureone Accessories Compatible for Quest 2 ...,[],"[Video Games, PC, Virtual Reality, Headsets]",20.99,5219,1238
2,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0B7LV3DN2,4.0,1617641445475,Cell Phones & Accessories,Eyglo Replacement Elite Strap for Oculus/Meta ...,[],"[Video Games, PC, Virtual Reality, Headsets]",28.49,5219,3858
3,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B09WMQ6DXG,5.0,1620231368468,All Electronics,"AMVR Controller Grips for Quest 2, Controller ...",[],"[Video Games, PC, Virtual Reality, Controllers]",15.99,5219,1276
4,AFFZVSTUS3U2ZD22A2NPZSKOCPGQ,B01GW3LRD2,5.0,1491589434000,Video Games,Forza Horizon 3 – Xbox One,[You’re in charge of the Horizon Festival. Cus...,"[Video Games, Xbox One, Games]",22.0,1305,2448
...,...,...,...,...,...,...,...,...,...,...,...
15252,AEUDRWR7PNA6JPXZ5RTN6KCXKOBA,B0BK673BF4,5.0,1636340286500,Computers,HK GAMING GK61 Mechanical Gaming Keyboard 60 P...,[],"[Video Games, PC, Accessories, Gaming Keyboards]",59.99,3947,5277
15253,AFEYPRZTCVN4WURYQOEYNTD2JFYQ,B000N5Z2L4,5.0,1648670003849,Video Games,Xbox Live Gold: 1 Month Membership [Digital Code],"[""Experience the rush of competitive and coope...","[Video Games, Xbox Digital Content, Subscripti...",9.99,4970,2621
15254,AHPWCVRF23GACCNGHT6VRIPFTFFA,B0B5SV7L99,5.0,1631554680324,Video Games,Streets of Rage 4 - Xbox One,"[Streets of Rage, considered One of the best b...","[Video Games, Xbox One, Games]",34.99,5883,957
15255,AGRQ2ELNB47RERPPMRDMXK7EOGZA,B0771371PM,5.0,1630392097798,Video Games,"Hyperkin ""Scout"" Premium Controller for SNES","[The Hyperkin ""Scout"" Premium Controller for S...","[Video Games, Legacy Systems, Nintendo Systems...",14.99,10840,2428


In [19]:
batch_size = 128

dataset = ItemSequenceDataset(
    full_df,
    'user_index',
    'item_index',
    'rating',
    'timestamp',
    val_timestamp=val_timestamp,
    is_train=True
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

val_dataset = ItemSequenceDataset(
    full_df,
    'user_index',
    'item_index',
    'rating',
    'timestamp',
    val_timestamp=val_timestamp,
    is_train=False
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [20]:
full_df.loc[lambda df: df['user_id'].eq('AGS4TR4K5DMBRAFNBYSB2I2RCHHQ')]

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price,user_index,item_index
7765,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B01MCXS6ZK,5.0,1498792892031,All Electronics,ButterFox BRENDO Nintendo 2DS Hard Case with 2...,"[Light yet durable with a soft lining, this co...","[Video Games, Legacy Systems, Nintendo Systems...",18.88,8921,3284
7766,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B07PB2DYTT,1.0,1556683035336,All Electronics,RGEEK FreeMcBoot FMCB 1.966 PS2 Memory Card 64...,[],"[Video Games, Legacy Systems, PlayStation Syst...",14.99,8921,758
1,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B0936HDGJ6,5.0,1652494657651,All Electronics,ISENPENK Switch Controller for Nintendo with W...,[],"[Video Games, Nintendo Switch, Accessories, Co...",21.99,8921,1472
4665,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B07RN8JGZB,1.0,1632141542619,Computers,2PACK PS2 Controller Extension Cable Cord 6FT/...,[This extension cable is the perfect choice fo...,"[Video Games, Legacy Systems, PlayStation Syst...",8.99,8921,3742
8670,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B077G9B628,1.0,1653270599580,Video Games,Xenoblade Chronicles 2 (Nintendo Switch),"[Product Description, Explore an endless ocean...","[Video Games, Nintendo Switch, Games]",55.7,8921,625


In [21]:
def get_last_item(user_indice: int, timestamp: int):
    return val_dataset.get_item_sequence(user_indice, timestamp)[-1]

In [22]:
output = {
    "user_indice": [],
    "user_last_item_indice": [],
    "recommendation": [],
    "score": [],
}
for i, row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
    user_indice = idm.get_user_index(row.user_id)
    last_item = get_last_item(user_indice, row.timestamp)
    recommendations = model.recommend([last_item], k=100, progress_bar_type=None)
    output['user_indice'].extend([user_indice] * len(recommendations['item_indice']))
    output['recommendation'].extend(recommendations['recommendation'])
    output['score'].extend(recommendations['score'])
    output["user_last_item_indice"].extend(recommendations['item_indice'])

  0%|          | 0/15257 [00:00<?, ?it/s]

# Evaluate

## Ranking metrics

In [23]:
user_col = args.user_col
item_col = args.item_col
rating_col = args.rating_col

In [24]:
from src.eval import create_label_df, create_rec_df, merge_recs_with_target

In [25]:
recommendations_df = pd.DataFrame(output).pipe(create_rec_df, idm)
recommendations_df

,user_indice,user_last_item_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,12359,1034,4407,0.848228,1.0,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B08C2NZNC4
1,12359,1034,660,0.800187,2.0,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B07BFWLXLS
2,12359,1034,874,0.789441,3.0,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B0151K6J9Y
3,12359,1034,432,0.783675,4.0,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B00C9DL1HC
4,12359,1034,3095,0.781553,5.0,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B08T127X2B
...,...,...,...,...,...,...,...
1525695,7909,1984,4570,0.382859,296.0,AEEXD6AIYLX4YWPPQCM7M7PQUHKQ,B074FJYPJB
1525696,7909,1984,1313,0.382830,297.0,AEEXD6AIYLX4YWPPQCM7M7PQUHKQ,B00P1OYN0S
1525697,7909,1984,4153,0.381724,298.0,AEEXD6AIYLX4YWPPQCM7M7PQUHKQ,B01GW3H3U8
1525698,7909,1984,3267,0.381648,299.0,AEEXD6AIYLX4YWPPQCM7M7PQUHKQ,B08FCVCDTM


In [26]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
7329,AE2WIEN3VYPFDPZ3IATSPFQQV2FQ,B094YHB1QK,1.0,1.0
1285,AH4JY7JDGBM5KWZNNS564ZZ7KCEA,B07D53QSMK,5.0,1.0
9762,AH5XFYDNDNYOSEYC7VZ6ORVNS3UA,B07XHMFCJ2,5.0,1.0
4347,AG3N3EMFIFGW66WOIC4MW55IUS6Q,B0BXQH38S6,4.0,1.0
13990,AGJAX3U73M52QLRSR4VYWZXEENMA,B07ZJ6RY1W,5.0,1.0
...,...,...,...,...
3169,AETPWUNCW3KZDTKNUQGS5YYKNUEA,B08JTNL426,3.0,35.0
11389,AETPWUNCW3KZDTKNUQGS5YYKNUEA,B008CP6MA2,2.0,36.0
6118,AETPWUNCW3KZDTKNUQGS5YYKNUEA,B07BZP7HML,2.0,37.0
8126,AETPWUNCW3KZDTKNUQGS5YYKNUEA,B008FPMBNG,2.0,38.0


In [27]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_indice,user_last_item_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
23,4111.0,3788.0,155.0,0.798575,1,AE22LPCN47WUTHSG67R6SKN4A4MQ,B010N0XTTU,0,NaN
24,4111.0,3788.0,428.0,0.766849,2,AE22LPCN47WUTHSG67R6SKN4A4MQ,B016MLWTCM,0,NaN
75,4111.0,3788.0,2703.0,0.766063,3,AE22LPCN47WUTHSG67R6SKN4A4MQ,B08R3HZPX2,0,NaN
77,4111.0,3788.0,423.0,0.761077,4,AE22LPCN47WUTHSG67R6SKN4A4MQ,B08W9C5TSQ,0,NaN
97,4111.0,3788.0,126.0,0.757130,5,AE22LPCN47WUTHSG67R6SKN4A4MQ,B0C5JFJ6PN,0,NaN
...,...,...,...,...,...,...,...,...,...
1540245,4467.0,2621.0,1941.0,0.647455,97,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,B01NAZ49SI,0,NaN
1540278,4467.0,2621.0,259.0,0.647113,98,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,B07WW5ZQ4Q,0,NaN
1540222,4467.0,2621.0,939.0,0.647031,99,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,B001C58T5U,0,NaN
1540235,4467.0,2621.0,89.0,0.646716,100,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,B00OA306WU,0,NaN


### Visualize

In [28]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metrics import PrecisionTopKMetric
from evidently.metrics import RecallTopKMetric
from evidently.metrics import FBetaTopKMetric
from evidently.metrics import NDCGKMetric
from evidently.metrics import PersonalizationMetric
import warnings

warnings.filterwarnings(
    action='ignore',
    category=FutureWarning,
    module=r'evidently.metrics.recsys.precision_recall_k'
)

from src.viz import color_scheme

In [29]:
column_mapping = ColumnMapping(
    recommendations_type='rank',
    target=rating_col,
    prediction='rec_ranking',
    item_id=item_col,
    user_id=user_col
)

report = Report(metrics=[
    NDCGKMetric(k=args.top_k),
    RecallTopKMetric(k=args.top_K),
    PrecisionTopKMetric(k=args.top_k),
    FBetaTopKMetric(k=args.top_k),
    PersonalizationMetric(k=args.top_k),
], options=[color_scheme])

report.run(
    reference_data=None,
    current_data=eval_df,
    column_mapping=column_mapping
)

evidently_report_fp = f"{args.notebook_persist_dp}/evidently_report.html"
os.makedirs(args.notebook_persist_dp, exist_ok=True)
report.save_html(evidently_report_fp)

if args.log_to_mlflow:
    mlflow.log_artifact(evidently_report_fp)
    for metric_result in report.as_dict()['metrics']:
        metric = metric_result['metric']
        if metric == 'PersonalizationMetric':
            metric_value = float(metric_result['result']['current_value'])
            mlflow.log_metric(f"val_{metric}", metric_value)
            continue
        result = metric_result['result']['current'].to_dict()
        for kth, metric_value in result.items():
            mlflow.log_metric(f"val_{metric}_at_k_as_step", metric_value, step=kth)

/home/dvquys/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


# Predict

In [30]:
val_df.iloc[[2]]

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
2,AF7HTSEWIKYSP5D3ST4EZIUK6PJQ,B08F5T3F9Y,5.0,1644540517651,None,Quest 2 Elite Strap with Battery and Carrying ...,"[Designed for Meta Quest 2, this premium ergon...","[Video Games, Virtual Reality, Standalone Hard...",149.99


In [31]:
user_id = 'AFQAPWVESEJYTNZC23LDPQOH7QBA'
timestamp = 1630119475785
full_df.loc[lambda df: df['user_id'].eq(user_id)].sort_values('timestamp', ascending=False)

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price,user_index,item_index
5531,AFQAPWVESEJYTNZC23LDPQOH7QBA,B09MRM36JJ,5.0,1652125278115,Computers,Razer Gaming Mouse (2018 Model),[Gaming Mouse (2018 model)],"[Video Games, PC, Accessories, Gaming Mice]",29.99,3022,5240
4821,AFQAPWVESEJYTNZC23LDPQOH7QBA,B09Y5PNFT6,5.0,1634411930426,Computers,Corsair Virtuoso RGB Wireless Gaming Headset -...,[The CORSAIR Virtuoso RGB Wireless delivers a ...,"[Video Games, PC, Accessories, Headsets]",150.67,3022,2510
3,AFQAPWVESEJYTNZC23LDPQOH7QBA,B09GM4283G,5.0,1630119475785,Video Games,PlayStation PULSE 3D Wireless Headset – Midnig...,[Ignite your gaming nights with the ultra-slee...,"[Video Games, PlayStation 5, Accessories, Gami...",99.0,3022,3224
13623,AFQAPWVESEJYTNZC23LDPQOH7QBA,B07624RBWB,5.0,1622749151959,Video Games,Nintendo Switch Pro Controller,[],"[Video Games, Nintendo Switch, Accessories, Co...",69.0,3022,2023
13622,AFQAPWVESEJYTNZC23LDPQOH7QBA,B08JHX17ZZ,5.0,1621461425122,Video Games,Super Mario 3D World + Bowser's Fury - Nintend...,"[Power up, team up, one up to save the Sprixie...","[Video Games, Nintendo Switch, Games]",49.1,3022,1534
13621,AFQAPWVESEJYTNZC23LDPQOH7QBA,B0BZ8GZW38,5.0,1621461003075,Computers,Razer Huntsman Mini 60% Gaming Keyboard: Fast ...,[Dominate on a different scale with the Razer ...,"[Video Games, PC, Accessories, Gaming Keyboards]",97.1,3022,3646
13620,AFQAPWVESEJYTNZC23LDPQOH7QBA,B088MYMD21,5.0,1621460946569,Video Games,Nintendo Paper Mario: The Origami King - Switch,[A new paper-crafted Mario adventure unfolds o...,"[Video Games, Nintendo Switch, Accessories]",49.99,3022,791
13619,AFQAPWVESEJYTNZC23LDPQOH7QBA,B0C89J78ZW,5.0,1601752199803,Computers,Logitech G Pro Wireless Gaming Mouse & G PRO M...,[Logitech G Pro Wireless Gaming Mouse with Esp...,"[Video Games, PC, Accessories, Gaming Mice]",176.93,3022,4906


In [32]:
user_indice = idm.get_user_index(user_id)
user_last_item_indice = get_last_item(user_indice, 1630119475785)
user_last_item_indice

np.int64(2023)

In [33]:
item_id = 'B09GM4283G'
item_indice = idm.get_item_index(item_id)

model.predict([item_indice], [user_last_item_indice])

tensor([0.3191], device='cuda:0')

# Clean up

In [34]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/15 12:16:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run 013-content-based at: http://localhost:5003/#/experiments/1/runs/033447f1695f4a03bef60844b5ff51b4.
2024/09/15 12:16:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.
